<a href="https://colab.research.google.com/github/ajit-gvs/HCR/blob/master/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import tensorflow as tf
print(tf.__version__)

import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from PIL import Image
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
directory='/content/data-directory'







2.4.1


In [2]:
def image_binarization(image):
  # converting image to grayscale
  image_grayscale=image.convert('L')
  img=np.array(image_grayscale) 
   
  img[img <128] = 0
  img[img >=128] = 254 

  #making white as foreground pixels and black as background pixels
  img[img==0]=255
  img[img==254]=0
  
  return img


def line_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a line
  
  lines=[]
  begin_matrix=[]
  stop_matrix=[]
  del_start_matrix=[]
  del_end_matrix=[]

  horizontal_hist = np.sum(img,axis=1,keepdims=True)/255
  start_count=0

  for i in range(len(horizontal_hist)):

    if horizontal_hist[i]>0 and horizontal_hist[i-1]==0:
      start_count+=1
      start_matrix.append(i)

    if horizontal_hist[i]==0 and start_count>0 and horizontal_hist[i-1]>0:
      end_matrix.append(i)

  
  
  if len(start_matrix)==len(end_matrix):
    for i in range(len(start_matrix)):
      if end_matrix[i]-start_matrix[i]<20:
        del_start_matrix.append(i)
        del_end_matrix.append(i)
    for i in range(len(start_matrix)):
      count=0
      for j in range(len(del_start_matrix)):
        if i==del_start_matrix[j]:
          count=count+1
      if count==0:
        begin_matrix.append(start_matrix[i])
    for i in range(len(end_matrix)):
      count=0
      for j in range(len(del_end_matrix)):
        if i==del_end_matrix[j]:
          count=count+1
      if count==0:
        stop_matrix.append(end_matrix[i])
    
    for i in range(len(begin_matrix)):
      lines.append(img[begin_matrix[i]:stop_matrix[i],:])

      

      

  
  return lines




def word_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a word
  dissection_matrix=[]
  
  words=[]
  m,n=img.shape
  
  length=[]
  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  
  
  start_count=0
  
  
  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]>0 and vertical_hist[0][i-1]==0:
      start_count+=1
      start_matrix.append(i)
    if vertical_hist[0][i]==0 and start_count>0 and vertical_hist[0][i-1]>0:
      end_matrix.append(i)
  
  
  length_mag=0
  for i in range(len(start_matrix)):
    if i>0:
      length_mag=(start_matrix[i]-end_matrix[i-1])
      length.append(length_mag)

  max=np.max(length)
     
  avg=max/3

  dissection_matrix.append([start_matrix[0],end_matrix[0]])
  j=0
  for i in range(len(length)-1):
    
    if length[i]> avg:
      dissection_matrix.append([start_matrix[i+1],end_matrix[i+1]])
      j=j+1
      
    if length[i]<=avg:
      dissection_matrix[j][1]=end_matrix[i+1]
  for i in range(len(dissection_matrix)):
    words.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
  
  return words


def char_segmentation(img):
  start_matrix=[]
  dissection_matrix=[]
  delete_matrix=[]
  address_matrix=[]
  
  
  characters=[]
  m,n=img.shape

  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  start_matrix.append(0)

  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]<10:
      start_matrix.append(i)

  for i in range(len(start_matrix)-1):
    if start_matrix[i+1]-start_matrix[i]<10:
      delete_matrix.append(i)
  
  for i in range(len(start_matrix)):
    count=0
    for j in range(len(delete_matrix)):
      if  i==delete_matrix[j]:
        count=count+1
    if count==0:
      address_matrix.append(start_matrix[i])


  for i in range(len(address_matrix)-1):
    dissection_matrix.append([address_matrix[i],address_matrix[i+1]])
  for i in range(len(dissection_matrix)):
    characters.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
    
    
  return characters

In [3]:
def data_label():

  samples=[]
  labels=[]

 
  
 
  
  
  
  for filename in os.listdir(directory):
    count=0
    text=open('/content/lines_new.txt')
    image_orig=Image.open('/content/data-directory/'+filename,'r')
    img=image_binarization(image_orig)
    lines=line_segmentation(img)
    for line in text:
      line_Split = line.strip().split(" ")
      filename_split=filename.split('.')
      linesplit=line_Split[0].split("-")
      string= linesplit[0]+'-'+linesplit[1]
      if string==filename_split[0]:
          count=count+1
    if count==len(lines): 
      for i in range(len(lines)):
        try:
          words=word_segmentation(lines[i])
          text=open('/content/lines_new.txt')
          for line in text:
            lineSplit = line.strip().split(' ')
            filename_split=filename.split('.')
            if lineSplit[0]==filename_split[0]+'-'+str(0)+str(i):
              word_split = lineSplit[8].split('|')
              if len(word_split)==len(words):
                for j in range(len(words)):
                  characters=char_segmentation(words[j])
                  if len(word_split[j])==len(characters):
                    for k in range(len(characters)):
                      
                      character=cv2.resize(characters[k],(96,96))
                      samples.append(character)
                    for l in word_split[j]:
                      labels.append(l) 

        except ValueError as ve:
          continue  
          
    
  
                 
  
  return  samples,labels

samples,labels=data_label()
print(len(samples))
print(len(labels))
for i in range(5):
  print(labels[i])


1506
1506
I
t
f
o
l


In [4]:
samples = np.array(samples, dtype='float' )/ 255.0
labels = np.array(labels)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels
samples.shape

(1506, 96, 96)

In [5]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(samples, labels, test_size=0.2, random_state=42)


In [6]:
len(testx)
testx.shape
trainx=trainx.reshape(trainx.shape[0],96,96,1)
testx=testx.reshape(testx.shape[0],96,96,1)


In [7]:
print(len(lb.classes_))
testy.shape
testx.shape


55


(302, 96, 96, 1)

In [ ]:
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'

In [8]:
pip install keras-tuner==1.0.0

     |████████████████████████████████| 92kB 3.5MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=316966825cab3dd2b4d5e1243975b8f54fbff3327ce71a0a0a638d6b8de71968
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [9]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:

from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation,Dropout

In [11]:
classes = len(lb.classes_)

In [12]:
HP_img_dims = (96,96,1)
def build(hp):
    classes = len(lb.classes_)
    model = keras.models.Sequential()
    # BLOCK1
    model.add(Conv2D(hp.Int('input_units',32,512,32),(3,3), 
                     input_shape=trainx.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout',[0.25,0.4,0.5])))
    

    for i in range(hp.Int("n_layers",1,6)):
      model.add(Conv2D(hp.Int(f'conv_{i}_units',32,512,32),(3,3)))
      model.add(Activation('relu'))
      model.add(Dropout(hp.Choice(f'dropout_{i}',[0.25,0.4,0.5])))

     
   
   

  

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    model.add(Dense(classes))
    model.add(Activation("softmax"))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
    

    

    return model
trainx=trainx.reshape(trainx.shape[0],96,96,1)
testx=testx.reshape(testx.shape[0],96,96,1)





In [ ]:
tuner=RandomSearch(
    build,objective='val_accuracy',
    max_trials=15,executions_per_trial=1)

tuner.search(x=trainx,
             y=trainy,
             epochs=5,
             batch_size=64,
             validation_data=(testx,testy))

INFO:tensorflow:Reloading Oracle from ./untitled_project/oracle.json


INFO:tensorflow:Reloading Oracle from ./untitled_project/oracle.json


Epoch 1/5
19/19 [==============================] - 196s 10s/step - loss: 3.6904 - accuracy: 0.1003 - val_loss: 2.9844 - val_accuracy: 0.1987
Epoch 2/5
19/19 [==============================] - 196s 10s/step - loss: 2.9151 - accuracy: 0.2455 - val_loss: 2.9215 - val_accuracy: 0.2152
Epoch 3/5
19/19 [==============================] - 206s 11s/step - loss: 2.7382 - accuracy: 0.2284 - val_loss: 2.9131 - val_accuracy: 0.2384
Epoch 4/5
19/19 [==============================] - 196s 10s/step - loss: 2.6579 - accuracy: 0.2680 - val_loss: 2.9678 - val_accuracy: 0.2616
Epoch 5/5
19/19 [==============================] - 196s 10s/step - loss: 2.3658 - accuracy: 0.3503 - val_loss: 2.9504 - val_accuracy: 0.2682


Epoch 1/5
19/19 [==============================] - 331s 17s/step - loss: 3.6574 - accuracy: 0.1036 - val_loss: 3.1562 - val_accuracy: 0.1921
Epoch 2/5
19/19 [==============================] - 329s 17s/step - loss: 2.9568 - accuracy: 0.1763 - val_loss: 2.9805 - val_accuracy: 0.2185
Epoch 3/5
19/19 [==============================] - 329s 17s/step - loss: 2.8028 - accuracy: 0.2200 - val_loss: 2.9349 - val_accuracy: 0.2119
Epoch 4/5
19/19 [==============================] - 329s 17s/step - loss: 2.6656 - accuracy: 0.2508 - val_loss: 2.9711 - val_accuracy: 0.2384
Epoch 5/5
19/19 [==============================] - 328s 17s/step - loss: 2.5142 - accuracy: 0.2510 - val_loss: 2.9675 - val_accuracy: 0.2517


Epoch 1/5
19/19 [==============================] - 383s 20s/step - loss: 203.2422 - accuracy: 0.0987 - val_loss: 3.1717 - val_accuracy: 0.0960
Epoch 2/5
19/19 [==============================] - 391s 21s/step - loss: 3.1976 - accuracy: 0.1044 - val_loss: 3.0503 - val_accuracy: 0.1358
Epoch 3/5
19/19 [==============================] - 374s 20s/step - loss: 3.1230 - accuracy: 0.1247 - val_loss: 3.0437 - val_accuracy: 0.1424
Epoch 4/5
19/19 [==============================] - 398s 21s/step - loss: 3.0931 - accuracy: 0.1305 - val_loss: 3.1112 - val_accuracy: 0.1358
Epoch 5/5
19/19 [==============================] - 373s 20s/step - loss: 3.1085 - accuracy: 0.1233 - val_loss: 3.0710 - val_accuracy: 0.1358


...

Epoch 1/5
19/19 [==============================] - 468s 25s/step - loss: 4.4705 - accuracy: 0.1033 - val_loss: 3.0896 - val_accuracy: 0.1424
Epoch 2/5
19/19 [==============================] - 467s 25s/step - loss: 3.1094 - accuracy: 0.1339 - val_loss: 2.9839 - val_accuracy: 0.1987
Epoch 3/5
19/19 [==============================] - 466s 25s/step - loss: 3.0121 - accuracy: 0.1748 - val_loss: 2.9578 - val_accuracy: 0.2086
Epoch 4/5
19/19 [==============================] - 469s 25s/step - loss: 2.6842 - accuracy: 0.2207 - val_loss: 2.9371 - val_accuracy: 0.2285
Epoch 5/5
19/19 [==============================] - 468s 25s/step - loss: 2.5050 - accuracy: 0.3005 - val_loss: 2.9594 - val_accuracy: 0.2285


Epoch 1/5
19/19 [==============================] - 779s 41s/step - loss: 3.4385 - accuracy: 0.1391 - val_loss: 3.1956 - val_accuracy: 0.2252
Epoch 2/5
19/19 [==============================] - 803s 41s/step - loss: 2.8337 - accuracy: 0.2157 - val_loss: 2.9602 - val_accuracy: 0.2351
Epoch 3/5
19/19 [==============================] - 802s 42s/step - loss: 2.6136 - accuracy: 0.2715 - val_loss: 2.9551 - val_accuracy: 0.2351
Epoch 4/5
19/19 [==============================] - 779s 41s/step - loss: 2.4191 - accuracy: 0.3081 - val_loss: 2.9470 - val_accuracy: 0.2616
Epoch 5/5
19/19 [==============================] - 781s 41s/step - loss: 2.0055 - accuracy: 0.4021 - val_loss: 3.0064 - val_accuracy: 0.2815


Epoch 1/5
19/19 [==============================] - 950s 49s/step - loss: 3.4934 - accuracy: 0.1312 - val_loss: 3.2114 - val_accuracy: 0.1921
Epoch 2/5
19/19 [==============================] - 937s 49s/step - loss: 2.9533 - accuracy: 0.2090 - val_loss: 3.2861 - val_accuracy: 0.2020
Epoch 3/5
19/19 [==============================] - 968s 50s/step - loss: 2.8171 - accuracy: 0.2006 - val_loss: 3.0318 - val_accuracy: 0.2185
Epoch 4/5
19/19 [==============================] - 942s 50s/step - loss: 2.6657 - accuracy: 0.2436 - val_loss: 3.0387 - val_accuracy: 0.2351
Epoch 5/5
19/19 [==============================] - 968s 51s/step - loss: 2.5501 - accuracy: 0.2696 - val_loss: 2.9392 - val_accuracy: 0.2219


Epoch 1/5
19/19 [==============================] - 458s 24s/step - loss: 3.4602 - accuracy: 0.1357 - val_loss: 2.9434 - val_accuracy: 0.2384
Epoch 2/5
19/19 [==============================] - 437s 23s/step - loss: 2.7512 - accuracy: 0.2439 - val_loss: 2.9108 - val_accuracy: 0.2318
Epoch 3/5
19/19 [==============================] - 434s 23s/step - loss: 2.5185 - accuracy: 0.2771 - val_loss: 2.8692 - val_accuracy: 0.2748
Epoch 4/5
19/19 [==============================] - 457s 23s/step - loss: 1.9576 - accuracy: 0.3972 - val_loss: 3.0203 - val_accuracy: 0.2715
Epoch 5/5
19/19 [==============================] - 458s 24s/step - loss: 1.3798 - accuracy: 0.5741 - val_loss: 3.4638 - val_accuracy: 0.2815


Epoch 1/5
19/19 [==============================] - 363s 19s/step - loss: 3.7540 - accuracy: 0.1065 - val_loss: 3.0558 - val_accuracy: 0.2020
Epoch 2/5
19/19 [==============================] - 381s 20s/step - loss: 3.0092 - accuracy: 0.1889 - val_loss: 2.9638 - val_accuracy: 0.2252
Epoch 3/5
15/19 [======================>.......] - ETA: 1:12 - loss: 2.8629 - accuracy: 0.2132

In [ ]:
HP_epoch = 1000
batch_size=16

HP_image_dims = (96,96,1)


from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras import backend 
import keras
import keras.utils
from keras import utils as np_utils
from keras.optimizers import adam
from sklearn import metrics
from tensorflow.keras.callbacks import EarlyStopping
import statistics




HP_img_dims = (192,192,1)
HP_small_pattern = (2,2) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (3,3)



In [ ]:


  
class RacoonVGG:
  @staticmethod
  def build(height, width,depth, classes,dropout, neuronPct, neuronShrink,lr_rate):
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    
    small_pattern = (2,2) # UNCOMPRESSED or 1-2 compression IMAGES
    large_pattern = (3,3)
    model = Sequential()
    neuronCount = int(neuronPct * 5000)
    layer = 0
    count=0
    while neuronCount>25 and layer<10:
        # The first (0th) layer needs an input input_dim(neuronCount)
        if layer==0:
          model.add(Conv2D(int(neuronCount),HP_small_pattern, padding='same',
                     input_shape=input_shape))
          model.add(Activation('relu'))
          model.add(BatchNormalization(axis=channel_dim))
          model.add(MaxPooling2D(pool_size=HP_small_pattern))
          model.add(Dropout(dropout))
          neuronCount = neuronCount * neuronShrink
          layer += 1

        
        if layer>0 and layer<10:
          model.add(Conv2D(int(neuronCount),HP_small_pattern, padding='same'))
          model.add(Activation('relu'))
          model.add(BatchNormalization(axis=channel_dim))
          model.add(MaxPooling2D(pool_size=HP_large_pattern))
          model.add(Dropout(dropout))
          layer += 1
          neuronCount = neuronCount * neuronShrink
        
          
        
    




            
    model.add(Flatten())
    model.add(Dense(int(neuronCount)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(int(neuronCount)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    return model  

    
  

    

trainx=trainx.reshape(trainx.shape[0],192,192,1)
testx=testx.reshape(testx.shape[0],192,192,1)

In [ ]:
def evaluate_network(dropout, neuronPct, neuronShrink,lr_rate):
  epochs_needed = []
  mean_benchmark = []

  model = RacoonVGG.build(height=HP_img_dims[0], width=HP_img_dims[1],depth=HP_img_dims[2],classes = len(lb.classes_),
                          dropout=dropout, neuronPct=neuronPct, neuronShrink=neuronShrink,lr_rate=lr_rate
                          ) 
  opt = keras.optimizers.Adam(learning_rate=lr_rate)
  model.compile(loss='categorical_crossentropy', optimizer=opt)
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=100, verbose=0, mode='auto', restore_best_weights=True)
  hist = model.fit(aug.flow(trainx, trainy, batch_size=batch_size),
                 validation_data=(testx, testy),verbose=0,
                 steps_per_epoch= len(trainx) // batch_size,
                  epochs=5)
  epochs = monitor.stopped_epoch
  epochs_needed.append(epochs)
  
  
  score = model.evaluate(testx, testy)
  acc=(1/score)
  
  #mean_benchmark.append(score)
  #m1 = statistics.mean(mean_benchmark)
  keras.backend.clear_session()
  
  return acc
  

In [ ]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization


  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=f0fe0abff5e1d79d922e8836da192c9a8c0b4817d0ee096de402e1531959d5ef
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

pbounds = {'dropout': (0.0, 0.499),
           'neuronPct': (0.01, 1),
           'neuronShrink': (0.01, 1),
           'lr_rate': (0.0, 0.1),
          }

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1)

optimizer.maximize(init_points=4, n_iter=2)

print(optimizer.max)

|   iter    |  target   |  dropout  |  lr_rate  | neuronPct | neuron... |
-------------------------------------------------------------------------
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


756/756 [==============================] - 1s 987us/step
|  1        |  0.3012   |  0.2081   |  0.07203  |  0.01011  |  0.3093   |


ResourceExhaustedError: ignored